# get all reviews for each boutique in a single text:

In [1]:
''' Import libraries'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import string

from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.summarization.textcleaner import split_sentences

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE

from textblob import TextBlob, Word, Blobber
# to install textblob in your conda packages:
# 1. go to anaconda prompt
# 2. cd Anaconda3>Scripts>conda install -c conda-forge textblob
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')
from nltk import word_tokenize,sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\farnaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\farnaz\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\farnaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### step1: Define functions and classes to analyze reviews

1. To get the tfidf score for each boutique:

    1-1. get all the reviews for one boutique and add them together as a single string
    
    1-2. clean the string, remove all punctuations and stopwords
    
    1-3. create a function to give a matrix of tfidf_scores and vocabulary of terms
    
    
2. Plot the word-embedding of each boutique

## 1-1. get all the reviews for one boutique and add them together as a single string :

In [2]:
#to accelerate the process create a sample data frame to work on classes and functions instead of the huge original dataframe:

dict_CaryLane={'boutique_name':"Cary Lane", 'number_reviews':3, 'reviews':["I popped in here randomly the other day and was SO surprised! They had several of my favorite designers that I tend to shop online for through Revolve. Everything is brand new and heavily discounted...its like going into the MOST organized designer sample sale ever. I dont know how they do it, but so glad I discovered them! Highly recommend. Its not just one offs either...they have multiple sizes in everything!",
         "Are you tired of paying too much money for fashion and quality clothing. You will be amazed when you walk in this store. They great when you walk in. The merchandise selection is fantastic. Plus if you pay cash you get 10% off.",
         "Let just say I emailed the  owner stating I had no receipt and I made with cash. I'm still waiting several weeks later for what are my options."], 'star_ratings':[5,5,2]}

dict_SanFran={"boutique_name":"San Fran","number_reviews":4, "reviews":["Whenever I come to San Francisco I always look out for cool local shops. This is one of them. Got a really cool hat and a mug. Anytime I'm able to buy a non typical souvenir, I'm happy!",
              "Very, very cool place. The T-shirt's are very soft, and I really liked the designs. Bought a tie dye onesie and a beanie for my granddaughter-on-the-way.  The staff was bright and cheery. I highly recommend this place.", 
              "The staff here at the cafe and ice cream section generally can't be bothered to pretend to care about their customers.", 
              "The staff that is not to bother with really downer vibes. The clothing is not worth the price tag. And in overall the premise of what the brand once was is completely gone. Completely avoid this place"], "star_ratings":[5,3,2,4]}


In [3]:
# display.max_colwidth sets the maximum width of columns. Cells of this length or longer will be truncated with an ellipsis.
pd.options.display.max_colwidth = 1000

### exploring different dataframes and functions:

### Type 1

In [4]:
df_review1 = pd.DataFrame(dict_CaryLane)
df_review2 = pd.DataFrame(dict_SanFran)
coulmns = ["boutique_name","number_reviews", "star_ratings","reviews"]
df_review = pd.concat([df_review1,df_review2], ignore_index=False)
df_review

#Convert date to datetime type before adding to dataframe:
#df_review['date'] = df_review['date'].apply(lambda x: pd.to_datetime(x[0]))
#'date':[12/20/2019,10/22/2019,1/6/2020], 'date':[8/7/2018, 6/17/2017, 9/9/2018, 2/23/2020]

,boutique_name,number_reviews,reviews,star_ratings
0,Cary Lane,3,"I popped in here randomly the other day and was SO surprised! They had several of my favorite designers that I tend to shop online for through Revolve. Everything is brand new and heavily discounted...its like going into the MOST organized designer sample sale ever. I dont know how they do it, but so glad I discovered them! Highly recommend. Its not just one offs either...they have multiple sizes in everything!",5
1,Cary Lane,3,Are you tired of paying too much money for fashion and quality clothing. You will be amazed when you walk in this store. They great when you walk in. The merchandise selection is fantastic. Plus if you pay cash you get 10% off.,5
2,Cary Lane,3,Let just say I emailed the owner stating I had no receipt and I made with cash. I'm still waiting several weeks later for what are my options.,2
0,San Fran,4,"Whenever I come to San Francisco I always look out for cool local shops. This is one of them. Got a really cool hat and a mug. Anytime I'm able to buy a non typical souvenir, I'm happy!",5
1,San Fran,4,"Very, very cool place. The T-shirt's are very soft, and I really liked the designs. Bought a tie dye onesie and a beanie for my granddaughter-on-the-way. The staff was bright and cheery. I highly recommend this place.",3
2,San Fran,4,The staff here at the cafe and ice cream section generally can't be bothered to pretend to care about their customers.,2
3,San Fran,4,The staff that is not to bother with really downer vibes. The clothing is not worth the price tag. And in overall the premise of what the brand once was is completely gone. Completely avoid this place,4


In [5]:
#get_all_reviews1 function: ???????????????????????????????????????????????????????????????????????????????????????????????
'''adds all reviews related to one boutique to a single string '''

def get_all_review1(boutique_names, df_reviews): 
    for boutique in boutique_names:
        doc = [] 
        sub = df_reviews.loc[df_reviews['boutique_name'] == boutique]
        for index, row in sub.iterrows():
            doc.append(row["reviews"])
    print("boutique_name:", boutique, "\n")
    print("reviews_summary:", doc,"\n" )
    
    
boutique_names=df_review["boutique_name"]
#print(boutique_names)
doc = get_all_review1(boutique_names, df_review)
doc

boutique_name: San Fran 

reviews_summary: ["Whenever I come to San Francisco I always look out for cool local shops. This is one of them. Got a really cool hat and a mug. Anytime I'm able to buy a non typical souvenir, I'm happy!", "Very, very cool place. The T-shirt's are very soft, and I really liked the designs. Bought a tie dye onesie and a beanie for my granddaughter-on-the-way.  The staff was bright and cheery. I highly recommend this place.", "The staff here at the cafe and ice cream section generally can't be bothered to pretend to care about their customers.", 'The staff that is not to bother with really downer vibes. The clothing is not worth the price tag. And in overall the premise of what the brand once was is completely gone. Completely avoid this place'] 



In [6]:
#get_all_reviews2 function:
'''adds all reviews related to one boutique to a single string '''

def get_all_review2(boutique_names, df_reviews): 
    for boutique in boutique_names:
        print("boutique_name:", boutique, "\n")
        doc = []
        sub = df_reviews.loc[df_reviews['boutique_name'] == boutique]
        for index, row in sub.iterrows(): 
            doc.append(row["reviews"])
        print("reviews_summary:", doc,"\n" )
        
    
boutique_names=df_review["boutique_name"]
#print(boutique_names)
doc = get_all_review2(boutique_names, df_review)
doc

boutique_name: Cary Lane 

reviews_summary: ['I popped in here randomly the other day and was SO surprised! They had several of my favorite designers that I tend to shop online for through Revolve. Everything is brand new and heavily discounted...its like going into the MOST organized designer sample sale ever. I dont know how they do it, but so glad I discovered them! Highly recommend. Its not just one offs either...they have multiple sizes in everything!', 'Are you tired of paying too much money for fashion and quality clothing. You will be amazed when you walk in this store. They great when you walk in. The merchandise selection is fantastic. Plus if you pay cash you get 10% off.', "Let just say I emailed the  owner stating I had no receipt and I made with cash. I'm still waiting several weeks later for what are my options."] 

boutique_name: Cary Lane 

reviews_summary: ['I popped in here randomly the other day and was SO surprised! They had several of my favorite designers that I 

### Type 2

#### Manipulate Review DataFrame for easy access to required data if necessary:

In [27]:
#review related dataframe will contain "number_reviews", "star_ratings", "reviews" and "boutique_name" information.
''' 
#write a function to separate all the related info for each boutiques:
#1.group dataframe by each boutique_name using groupby method.
#2.use the average star_ratings of all reviews as the unique star_rating of the boutique.
#3.add all reviews of each boutique into one single list to analyse feedbacks, visualize results and create prediction models.
#4.the average number of reviews will remain the same for each boutique.
'''

def df_checklist(dataframe):
    #agg_function = {"number_reviews":lambda x: x.mean(), "star_ratings":lambda x: x.mean(), "reviews": lambda x:list(x)}
    agg_function = {"number_reviews":lambda x: x.mean(), "star_ratings":lambda x: x.mean(), "reviews": lambda x:x.sum()}
    #create DataFrameGroupBy object and conver it to dataframe using aggregate method:
    df = dataframe.groupby("boutique_name").aggregate(agg_function).reset_index()
    return df 

In [49]:
df = df_checklist(df_review)
df

,boutique_name,number_reviews,star_ratings,reviews
0,Cary Lane,3,4.0,"I popped in here randomly the other day and was SO surprised! They had several of my favorite designers that I tend to shop online for through Revolve. Everything is brand new and heavily discounted...its like going into the MOST organized designer sample sale ever. I dont know how they do it, but so glad I discovered them! Highly recommend. Its not just one offs either...they have multiple sizes in everything!Are you tired of paying too much money for fashion and quality clothing. You will be amazed when you walk in this store. They great when you walk in. The merchandise selection is fantastic. Plus if you pay cash you get 10% off.Let just say I emailed the owner stating I had no receipt and I made with cash. I'm still waiting several weeks later for what are my options."
1,San Fran,4,3.5,"Whenever I come to San Francisco I always look out for cool local shops. This is one of them. Got a really cool hat and a mug. Anytime I'm able to buy a non typical souvenir, I'm happy!Very, very cool place. The T-shirt's are very soft, and I really liked the designs. Bought a tie dye onesie and a beanie for my granddaughter-on-the-way. The staff was bright and cheery. I highly recommend this place.The staff here at the cafe and ice cream section generally can't be bothered to pretend to care about their customers.The staff that is not to bother with really downer vibes. The clothing is not worth the price tag. And in overall the premise of what the brand once was is completely gone. Completely avoid this place"


In [55]:
# test:
pd.options.display.max_colwidth = 1000
rows = df[(df.boutique_name == 'Cary Lane')]
reviews = rows.reviews.to_string()
# reviews = rows.reviews
reviews

"0    I popped in here randomly the other day and was SO surprised! They had several of my favorite designers that I tend to shop online for through Revolve. Everything is brand new and heavily discounted...its like going into the MOST organized designer sample sale ever. I dont know how they do it, but so glad I discovered them! Highly recommend. Its not just one offs either...they have multiple sizes in everything!Are you tired of paying too much money for fashion and quality clothing. You will be amazed when you walk in this store. They great when you walk in. The merchandise selection is fantastic. Plus if you pay cash you get 10% off.Let just say I emailed the  owner stating I had no receipt and I made with cash. I'm still waiting several weeks later for what are my options."

In [496]:
'''
input = list of boutique_names and the dataframe created for storing reviews
dataframe columns: "boutique_name", "number_reviews", "reviews", "star_rating" and "date" optional
output = a list of all reviews for each boutique_name
'''

def get_all_reviews3 (boutique_names, df_reviews):
    for boutique in boutique_names:
        row = df_reviews[(df_reviews.boutique_name == boutique)]
        print("boutique_name:", boutique, "\n")
        print("review_summary:", row.reviews,"\n")

        
        
boutique_names = ['CaryLane',"San Fran"]
df_reviews = df
all_reviews = get_all_reviews3 (boutique_names, df_reviews)
all_reviews

boutique_name: CaryLane 

review_summary: Series([], Name: reviews, dtype: object) 

boutique_name: San Fran 

review_summary: 1    [Whenever I come to San Francisco I always look out for cool local shops. This is one of them. Got a really cool hat and a mug. Anytime I'm able to buy a non typical souvenir, I'm happy!, Very, very cool place. The T-shirt's are very soft, and I really liked the designs. Bought a tie dye onesie and a beanie for my granddaughter-on-the-way.  The staff was bright and cheery. I highly recommend this place., The staff here at the cafe and ice cream section generally can't be bothered to pretend to care about their customers., The staff that is not to bother with really downer vibes. The clothing is not worth the price tag. And in overall the premise of what the brand once was is completely gone. Completely avoid this place]
Name: reviews, dtype: object 



### Type 3

In [266]:
#another version of dataframe to get the review lists instead of strings:
df_reviews = pd.DataFrame ([dict_CaryLane,dict_SanFran])
coulmns = ["boutique_name","number_reviews", "star_ratings","reviews"]
df_reviews

,boutique_name,number_reviews,reviews,star_ratings
0,Cary Lane,3,[I popped in here randomly the other day and w...,"[5, 5, 2]"
1,San Fran,4,[Whenever I come to San Francisco I always loo...,"[5, 3, 2, 4]"


In [499]:
#get_all_reviews4 function:
'''adds all reviews related to one boutique to a single string '''

def get_all_reviews4(boutique_names, df_reviews):
    doc = []
    for boutique in boutique_names:
        a = df_reviews.loc[df_reviews['boutique_name'] == boutique]
        reviews = a['reviews'].tolist()[0]
        doc.append(reviews)
    return doc


boutique_names=df_reviews["boutique_name"]
doc = get_all_reviews4(boutique_names, df_reviews)
doc

[['I popped in here randomly the other day and was SO surprised! They had several of my favorite designers that I tend to shop online for through Revolve. Everything is brand new and heavily discounted...its like going into the MOST organized designer sample sale ever. I dont know how they do it, but so glad I discovered them! Highly recommend. Its not just one offs either...they have multiple sizes in everything!',
  'Are you tired of paying too much money for fashion and quality clothing. You will be amazed when you walk in this store. They great when you walk in. The merchandise selection is fantastic. Plus if you pay cash you get 10% off.',
  "Let just say I emailed the  owner stating I had no receipt and I made with cash. I'm still waiting several weeks later for what are my options."],
 ["Whenever I come to San Francisco I always look out for cool local shops. This is one of them. Got a really cool hat and a mug. Anytime I'm able to buy a non typical souvenir, I'm happy!",
  "Ver